In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from apyori import apriori


In [2]:
import findspark
findspark.init('/opt/spark')
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
spark = SparkSession.builder\
            .appName('localiza')\
            .config("spark.executor.memory", "14g")\
            .config("spark.storage.memoryFraction", "0")\
            .config("spark.driver.memory", '14G') \
            .config("spark.executor.memory", '14G') \
            .config("spark.driver.maxResultSize", '4G') \
            .config("spark.executor.JavaOptions", "-XX:+UseG1GC")\
            .config("spark.executor.extraJavaOptions", "-XX:+UseCompressedOops -XX:ConcGCThreads=20 -XX:InitiatingHeapOcuupancyPercent=35")\
            .getOrCreate()

In [7]:
df = spark.read.csv('results_trab_all.csv', header=True, inferSchema=True)

In [8]:
df.printSchema()

root
 |-- suport: double (nullable = true)
 |-- confidence: double (nullable = true)
 |-- time: double (nullable = true)
 |-- frequents: string (nullable = true)
 |-- rule_size: integer (nullable = true)
 |-- algorithm: string (nullable = true)
 |-- itemset: string (nullable = true)
 |-- rules: string (nullable = true)



In [22]:
pd = df.filter('confidence == 0.1').orderBy(f.desc('suport')).select('suport', 'time', 'algorithm').toPandas()

In [23]:
pd

,suport,time,algorithm
0,0.10,0.078467,apriori
1,0.10,0.078364,fp_growth
2,0.09,0.069808,apriori
3,0.09,0.072838,fp_growth
4,0.08,0.111966,apriori
5,0.08,0.076647,fp_growth
6,0.07,0.111642,apriori
7,0.07,0.081026,fp_growth
8,0.06,0.123971,apriori
9,0.06,0.084204,fp_growth


In [27]:
import plotly.graph_objects as go


fig = go.Figure()

fig.add_trace(go.Scatter(
    x=pd[pd['algorithm'] == 'apriori']['suport'],
    y=pd[pd['algorithm'] == 'apriori']['time'],
    name = 'Apriori', # Style name/legend entry with html tags
))
fig.add_trace(go.Scatter(
    x=pd[pd['algorithm'] == 'apriori']['suport'],
    y=pd[pd['algorithm'] == 'fp_growth']['time'],
    name = 'FP-Growth', # Style name/legend entry with html tags
))
fig.show()